# Detecção de Vírus usando CNN (Convolution Neural Networks)

In [1]:
# bibliotecas
import sys
!{sys.executable} -m pip install numpy 
!{sys.executable} -m pip install sklearn 
!{sys.executable} -m pip install seaborn

In [2]:
import sys
import os
from math import log
import numpy as np
import pandas as pd
import scipy as sp
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
# pasta contendo o dataset de imagens de malwares para treinamento
path_root = "./malimg_dataset/train" 

In [ ]:
#reconhece as imagens, define o tamanho em pixels delas e o tamanho do vetor batch
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

In [ ]:
# imagens com índices
batches.class_indices

In [ ]:
# tuplas de imagens e etiquetas -> mostra suas dimensões
imgs, labels = next(batches)

In [ ]:
imgs.shape

In [ ]:
labels.shape

In [ ]:
# plota imagens com rótulos no Jupyter-Notebook
def plots(ims, figsize=(20,30), rows=10, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = 10 # len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(0,50):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(list(batches.class_indices.keys())[np.argmax(titles[i])], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
plots(imgs, titles = labels)

In [ ]:
classes = batches.class_indices.keys()
perc = (sum(labels)/labels.shape[0])*100

In [ ]:
plt.xticks(rotation='vertical')
plt.bar(classes,perc)

In [ ]:
# divide o dataset para treinamento e teste -> mostra suas dimensões
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
# import keras 
# added tensorflow
from tensorflow.keras.models import Sequential
# Um modelo seqüencial é apropriado para uma pilha simples de camadas, onde cada camada possui exatamente 
# um tensor de entrada e um tensor de saída.
from tensorflow.keras import Input #the .models has been removed 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
# Dense: implementa a operação: output = output = activation(dot(input, kernel) + bias) onde a ativação 
# é a função de ativação do elemento passada como argumento de ativação, o kernel é uma matriz de pesos 
# criada pela camada, e o viés é um vetor de viés criado por A camada (apenas aplicável se use_bias for verdadeira).
#
# Dropout: A camada de 'Dropout' define aleatoriamente unidades de entrada para 0 com uma frequência de taxa em cada 
# etapa durante o tempo de treinamento, o que ajuda a evitar o excesso de ajuste. As entradas não definidas como 0 
# são ampliadas em 1/(1 - taxa), de modo que a soma em todas as entradas permaneça inalterada.
#
# Flatten: Achata a entrada. Não afeta o tamanho do 'batch'.
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# Conv2D: Essa camada cria um kernel de convolução que é convoluído com a entrada da camada para produzir 
# um tensor de saídas. Se use_bias for verdadeiro, um vetor de polarização será criado e adicionado às saídas. 
# Finalmente, se a ativação não for nenhuma, ela também é aplicada às saídas.
#
# MaxPooling2D: As amostras reduzem a entrada ao longo de suas dimensões espaciais (altura e largura), assumindo o 
# valor máximo em uma janela de entrada (do tamanho definido por pool_size) para cada canal da entrada. A janela 
# é deslocada pelos avanços ao longo de cada dimensão.
from tensorflow.keras.layers import BatchNormalization

In [ ]:
num_classes = 25

In [ ]:
def malware_model(): #definindo a função 
    #modelo sequencial incremental (add())
    # Modelo sequential é ultilizado para uma pilha de camadas simples (cada camada possui um tensor de entrada e um de saida).
    Malware_model = Sequential()
    #conv2-> cria um kernel de convolução que é convoluído com a entrada da camada para produzir um tensor de saídas(convulução espacial sobre imagens)
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),#kernel_size -> especificando a altura e a largura da janela de convolução 2D. 
                     activation='relu',                             
                     input_shape=(64,64,3)))#64x64 RGB imagens
    #input_shape-> para transforma-lo em uma camada model
    #Input, ultilizado para exibir o um resumo do modelo desde o inicio 
    Malware_model.add(MaxPooling2D(pool_size=(2, 2))) #MaxPool-> reduz a resolução da entrada
    #ao longo das dimensões espaciais(altura e largura)
    #tomando o valor do pool_Size -> tamanho da janela sobre a qual se deve obter o max na janela de entrada.
    Malware_model.add(Conv2D(15, (3, 3), activation='relu'))
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    Malware_model.add(Dropout(0.25))
    Malware_model.add(Flatten())
    #flatten() sempre retorna uma cópia do array de entrada, ou seja, quaisquer alterações feitas no array 
    #retornado não modificarão o array original.
    Malware_model.add(Dense(128, activation='relu')) #camada para o construtor sequencial
    #Relu -> produz resultado no intervalo de 0(valores negativos) a infinito.Como seu resultado é 0 para valores negativos, 
    #tende a “apagar” alguns neurônios durante um passo forward, o que aumenta a velocidade do treinamento, 
    #mas por outro pode fazer com que esses neurônios “morram” e não aprendam nada se eles só receberem valores negativos.
    #Além disso, ela pode produzir ativações explodidas, já que não possui um limite positivo.
    Malware_model.add(Dropout(0.5)) #dropout aplicado ao neurinios ocultos do modelo de rede com uma taxa de 5%
    #Dropout é usado apenas durante o treinamento de um modelo e não é usado ao avaliar a habilidade do modelo.Seleciona 
    #aleatoriamente os nós que serão descatados com uma probabilidade proposta.
    Malware_model.add(Dense(50, activation='relu')) #camada para o construtor sequencial
    Malware_model.add(Dense(num_classes, activation='softmax')) # camada para o construtor sequencial
    #sofmax-> força a saida da RN e apresenta a possibilidade dos dados serem de classes definidas.
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    #categorical_crossentropy-> usada para tarefas de classificação,pode ser considerado como pertencente a uma categoria 
    #específica com probabilidade 1 e a outras categorias com probabilidade 0.
    #Adam Optimizer-> supera o restante do otimizador por uma margem considerável em termos de custo de treinamento (baixo)
    #e desempenho (alto). 
    #metrics-> é uma função usada para avaliar o desempenho do seu modelo.Semelhantes às funções de perda, exceto que os
    #resultados da avaliação de uma métrica não são usados ao treinar o modelo
    return Malware_model #retornando a função

In [ ]:
Malware_model = malware_model()

In [ ]:
#sumary () é chamado após o "modelo" ser contruido para exibir o contéudo
Malware_model.summary()

In [ ]:
y_train.shape #->treinando as matrize Y em sua respectiva dimensões.

In [ ]:
# O uso do fluxo de depuração de add+summary ->permite controlar a forma de como 
#uma pilha de Conv2D e MaxPooling 2D  